In [1]:
import pandas as pd
import numpy as np

In [3]:
#import icd10 data from both files
icd10_1 = pd.read_csv("icd10-1") #first half
icd10_2 = pd.read_csv("icd10-2") #second half

C:\Users\ynot3\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\ynot3\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning:

Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
##population add on per WHO population data file
pop = pd.read_csv('pop')

In [5]:
pop.head()

,Country,Admin1,SubDiv,Year,Sex,Frmat,Pop1,Pop2,Pop3,Pop4,...,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26,Lb
0,1060,NaN,NaN,1980,1,7,137100.0,3400.0,15800.0,NaN,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,NaN,NaN,1980,2,7,159000.0,4000.0,18400.0,NaN,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,NaN,NaN,1955,1,2,5051500.0,150300.0,543400.0,NaN,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,NaN,NaN,1955,2,2,5049400.0,145200.0,551000.0,NaN,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,NaN,NaN,1956,1,2,5353700.0,158700.0,576600.0,NaN,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0


In [ ]:
## read in the file with the cause mapping from 1-20
conCauses = pd.read_csv("condensed_causes_full.csv", low_memory = False)

In [ ]:
conCauses.tail()

In [ ]:
## copy the conCauses df into a merger df so the merge is simpler...will drop a bunch of columns
conCause_merger = conCauses
conCause_merger.tail()

In [ ]:
##merging df ready
conCause_merger.drop(conCause_merger.loc[:,'Frmat':'IM_Deaths4'], axis=1, inplace = True)
conCause_merger.drop(['Admin1','SubDiv','List'],axis =1, inplace = True)

In [ ]:
conCause_merger = conCause_merger.groupby(['new_cause','Country','Year','Cause']).sum().reset_index()
conCause_merger.tail()

In [ ]:
##pop1 is the total column so dropping all data except total pop year and country... sex is handled below
pop.drop(pop.loc[:,'Pop2':'Lb'],axis=1,inplace =True)
pop.drop(['Admin1','SubDiv','Frmat'],axis =1,inplace = True)

In [ ]:
## a population add on with total population by country by year.. the groupby.sum() combines male and female
pop = pop.groupby(['Country','Year']).sum().reset_index()

In [ ]:
pop.tail()

In [ ]:
#Combine ICD datasets into one and reset index 
icd10 = icd10_1.append(icd10_2).reset_index()

In [ ]:
#drop any na values in all cause mortality (turns out there are none)
icd10 = icd10[pd.notnull(icd10['Deaths1'])]

In [ ]:
#drop ages by death in favor of using all cause mortality the Death1 field
icd10.drop(icd10.loc[:, 'Deaths2':'Deaths26'], axis=1, inplace = True)

In [ ]:
#drop additional mortality formatting for location/age
icd10.drop(['index','Admin1','SubDiv','Frmat','IM_Frmat','IM_Deaths1','IM_Deaths2','IM_Deaths3','IM_Deaths4'], axis = 1, inplace = True)

In [ ]:
## both ICD databases combined with rows described above dropped. should have country,year,list,cause,sex and death1 remaining
icd10.head()

In [ ]:
#Here is some  basic EDA 
icd10.dtypes



In [ ]:
icd10.describe()

In [ ]:
##created a couple copys of the ICD10 data for more eda
##eda = icd10[icd10.List == 101].copy()
eda = icd10.copy()


In [ ]:
eda_count = eda.groupby(['List']).count().reset_index()

In [ ]:
## support for why we filtered on only the 104 code
eda_count

In [ ]:
## pulls only the 104 tagged lines from our main dataset
list_104 = icd10[icd10.List == 104].copy().reset_index()

In [ ]:
list_104.drop(['index'],axis =1,inplace = True)

In [ ]:
list_104.head()

In [ ]:
## this at this point each country,year,cause combo should have 1 summed number in Deaths1 - essentially removing sex
list_104_grouped = list_104.groupby(['Country','Year','Cause']).sum().reset_index()

In [ ]:
list_104_grouped.tail()

In [ ]:
## need to merge the population data into the 104 data to norm the death amounts to be able to identify outliers and run models
normed_104 = list_104_grouped.merge(pop[['Country','Year','Pop1']],)

In [ ]:
normed_104.tail()

In [ ]:
## this creates a new column that is % of deaths per total population in each cause
normed_104['Normed_Deaths'] = normed_104['Deaths1'] / normed_104['Pop1']

In [ ]:
normed_104.tail()

In [ ]:
## merging in cosolidated cause mapping
##normed_104 = list_104_grouped.merge(pop[['Country','Year','Pop1']],)
normed_merged = normed_104.merge(conCause_merger[['Country','Year','Cause','new_cause']],)

In [ ]:
normed_merged.tail()

In [ ]:
final_data = normed_merged.groupby(['Country','Year','new_cause']).sum().reset_index()

In [ ]:
## renaming new_cause to category and dropping excss columns sex,deaths1 and pop1
## this should be our final df ready for analysis may rearrange further depending on how we want to model
final_data.rename(columns={'new_cause':'Category'}, inplace=True)
final_data.drop(['Sex','Deaths1','Pop1'],axis=1, inplace = True)

In [1]:
final_data

NameError: name 'final_data' is not defined

In [ ]:
cluster = KMeans(n_clusters = 3)

In [ ]:
pivot_final_2015["cluster"] = cluster.fit_predict(pivot_final_2015[pivot_final_2015.columns[2:]])

In [ ]:
pivot_final_2015.head()

In [ ]:
# Principal component separation to create a 2-dimensional picture
pca = PCA(n_components = 2)
pivot_final_2015['x'] = pca.fit_transform(pivot_final_2015[categories])[:,0]
pivot_final_2015['y'] = pca.fit_transform(pivot_final_2015[categories])[:,1]
pivot_final_2015 = pivot_final_2015.reset_index()

In [ ]:
country_clusters_2015 = pivot_final_2015[["Country","cluster","x","y"]]

In [ ]:
country_clusters_2015.tail()

In [ ]:
#plotly scatterplot of clusters
trace0 = go.Scatter(x = country_clusters_2015[country_clusters_2015.cluster == 0]["x"],
                    y = country_clusters_2015[country_clusters_2015.cluster == 0]["y"],
                    name = "Cluster 1",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(15, 152, 152, 0.5)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace1 = go.Scatter(x = country_clusters_2015[country_clusters_2015.cluster == 1]["x"],
                    y = country_clusters_2015[country_clusters_2015.cluster == 1]["y"],
                    name = "Cluster 2",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(180, 18, 180, 0.5)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace2 = go.Scatter(x = country_clusters_2015[country_clusters_2015.cluster == 2]["x"],
                    y = country_clusters_2015[country_clusters_2015.cluster == 2]["y"],
                    name = "Cluster 3",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(132, 132, 132, 0.8)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))

data = [trace0, trace1, trace2]
fig = go.Figure(data = data)
fig.show()

In [ ]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", title="A Plotly Express Figure")

# If you print the figure, you'll see that it's just a regular figure with data and layout
# print(fig)

fig.show()